In [20]:
import os
import sys
import urllib, io

import numpy as np
import scipy.stats as stats
import pandas as pd

import pymongo as pm
from collections import Counter
import json
import re
import ast

from PIL import Image, ImageOps, ImageDraw, ImageFont 

from io import BytesIO
import base64

import  matplotlib
from matplotlib import pylab, mlab, pyplot
%matplotlib inline
from IPython.core.pylabtools import figsize, getfigs
plt = pyplot
import matplotlib as mpl
mpl.rcParams['pdf.fonttype'] = 42

import seaborn as sns
sns.set_context('talk')
sns.set_style('darkgrid')

from IPython.display import clear_output

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", message="numpy.dtype size changed")
warnings.filterwarnings("ignore", message="numpy.ufunc size changed")

In [21]:
## directory & file hierarchy
proj_dir = os.path.abspath('..')
datavol_dir = os.path.join(proj_dir,'data')
analysis_dir = os.path.abspath(os.path.join(os.getcwd(),'..'))
results_dir = os.path.join(proj_dir,'results')
plot_dir = os.path.join(results_dir,'plots')
csv_dir = os.path.join(results_dir,'csv')
json_dir = os.path.join(results_dir,'json')
exp_dir = os.path.abspath(os.path.join(proj_dir,'experiments'))
png_dir = os.path.abspath(os.path.join(datavol_dir,'png'))
will_dir = os.getcwd()

## add helpers to python path
if os.path.join(proj_dir,'stimuli') not in sys.path:
    sys.path.append(os.path.join(proj_dir,'stimuli'))
    
if not os.path.exists(results_dir):
    os.makedirs(results_dir)
    
if not os.path.exists(plot_dir):
    os.makedirs(plot_dir)   
    
if not os.path.exists(csv_dir):
    os.makedirs(csv_dir)       
    
import blockworld_helpers as utils    
    
# ## Assign variables within imported analysis helpers
# import analysis_helpers as h
# if sys.version_info[0]>=3:
#     from importlib import reload
# reload(h)

ModuleNotFoundError: No module named 'blockworld_helpers'

In [22]:
## set vars 
auth = pd.read_csv(os.path.join(analysis_dir,'auth.txt'), header = None) # this auth.txt file contains the password for the sketchloop user
pswd = auth.values[0][0]
user = 'sketchloop'
host = 'cogtoolslab.org' ## cocolab ip address

## have to fix this to be able to analyze from local
import pymongo as pm
conn = pm.MongoClient('mongodb://sketchloop:' + pswd + '@127.0.0.1')
db = conn['block_construction']
coll = db['silhouette']

## which iteration name should we use?
iterationName = 'testing'

## Sanity Checks

In [23]:
# Ensure one to one gameID and workerId 
# Should only happen if a repeat worker gets through

query = coll.find({"$and":[
                        {'workerId':{'$exists':True}},
                        {'condition':{'$ne':'practice'}},
                        {'eventType':'trial_end'},
                        {"$or":[{'iterationName':'pilot3'},
                                {'iterationName':'pilot2'}]},
                        {'trialNum':0}]
                     })

df_trial_end_full = pd.DataFrame(list(query.sort('timeAbsolute')))
#df_trial_end_full[['workerId','gameID']]


assert (np.mean(df_trial_end_full['workerId'].value_counts()) == np.mean(df_trial_end_full['gameID'].value_counts()))

In [24]:
# get ids of people with trial 15 data
query = coll.find({"$and":[
                        {'condition':{'$ne':'practice'}},
                        {'eventType':'trial_end'},
                        {'iterationName':'pilot3'},
                        {'trialNum':15}]
                     })
complete_data_df = pd.DataFrame(query)
complete_data_ids = list(complete_data_df['workerId'])

## Collect data from db and filter with sanity checks

In [25]:
query = coll.find({"$and":[
                        {'condition':{'$ne':'practice'}},
                        {'eventType':'trial_end'},
                        {'iterationName':'pilot3'}]
                     })

df_trial_end_full = pd.DataFrame(list(query.sort('timeAbsolute')))



# filter dataframe for complete datasets
df_trial_end_full_filtered = df_trial_end_full[df_trial_end_full.workerId.isin(complete_data_ids)]



# reduce to crucial information
df_trial_end_reduced_filtered = df_trial_end_full_filtered[['gameID','trialNum','phase','condition','eventType','score','normedScore','numBlocks','timeAbsolute','timeRelative','buildTime','currBonus','exploreResets','buildResets','allVertices','nPracticeAttempts','exploreStartTime','buildStartTime','buildFinishTime']]

df_for_analysis = df_trial_end_reduced_filtered.sort_values(by=['gameID', 'timeAbsolute'])
df_for_analysis


,gameID,trialNum,phase,condition,eventType,score,normedScore,numBlocks,timeAbsolute,timeRelative,buildTime,currBonus,exploreResets,buildResets,allVertices,nPracticeAttempts,exploreStartTime,buildStartTime,buildFinishTime
0,0335-390fd775-9a17-4e03-97de-fd31036a393d,0,build,mental,trial_end,0.00,0.412440,5,1.575400e+12,296387.370,60002,0.00,0,0,"[[{'x': 274.6545311714962, 'y': 592.6054374273...",None,1.575400e+12,1.575400e+12,1.575400e+12
1,0335-390fd775-9a17-4e03-97de-fd31036a393d,1,build,physical,trial_end,0.00,0.297015,3,1.575400e+12,392705.090,60003,0.00,0,0,"[[{'x': 232.00173028960907, 'y': 592.491661689...",None,1.575400e+12,1.575400e+12,1.575400e+12
2,0335-390fd775-9a17-4e03-97de-fd31036a393d,2,build,physical,trial_end,0.01,0.704637,7,1.575400e+12,489311.205,60002,0.01,0,0,"[[{'x': 277.97962409395717, 'y': 592.603205599...",None,1.575400e+12,1.575400e+12,1.575400e+12
5,0335-390fd775-9a17-4e03-97de-fd31036a393d,3,build,mental,trial_end,0.01,0.528537,6,1.575400e+12,585043.215,60003,0.00,0,0,"[[{'x': 245.94838550055294, 'y': 482.210460686...",None,1.575400e+12,1.575400e+12,1.575400e+12
10,0335-390fd775-9a17-4e03-97de-fd31036a393d,4,build,physical,trial_end,0.01,0.230413,7,1.575400e+12,683986.240,60022,0.00,0,0,"[[{'x': 307.0867862202134, 'y': 647.5002866143...",None,1.575400e+12,1.575400e+12,1.575400e+12
17,0335-390fd775-9a17-4e03-97de-fd31036a393d,5,build,mental,trial_end,0.01,0.576058,10,1.575400e+12,780304.605,60035,0.00,0,0,"[[{'x': 243.24081727850722, 'y': 592.618172179...",None,1.575400e+12,1.575400e+12,1.575400e+12
27,0335-390fd775-9a17-4e03-97de-fd31036a393d,6,build,mental,trial_end,0.01,0.538723,3,1.575400e+12,887315.325,60006,0.00,0,2,"[[{'x': 588.4935028127535, 'y': 482.4966114217...",None,1.575400e+12,1.575400e+12,1.575400e+12
40,0335-390fd775-9a17-4e03-97de-fd31036a393d,7,build,physical,trial_end,0.01,0.485845,6,1.575400e+12,1015019.930,60003,0.00,0,1,"[[{'x': 287.00110997225903, 'y': 482.496521752...",None,1.575400e+12,1.575400e+12,1.575400e+12
49,0335-390fd775-9a17-4e03-97de-fd31036a393d,8,build,mental,trial_end,0.01,0.452588,7,1.575400e+12,1109954.330,60002,0.00,0,0,"[[{'x': 320.75267178431494, 'y': 482.495202865...",None,1.575400e+12,1.575400e+12,1.575400e+12
58,0335-390fd775-9a17-4e03-97de-fd31036a393d,9,build,physical,trial_end,0.02,0.715155,9,1.575401e+12,1205356.035,60007,0.01,1,0,"[[{'x': 329.87132357347406, 'y': 592.688544902...",None,1.575400e+12,1.575400e+12,1.575401e+12


In [28]:
df_for_analysis.to_csv('block_silhouette_pilot3.csv')